<a href="https://colab.research.google.com/github/baihoneycutt/learn-py/blob/main/asyncio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1 Syntax and Native Coroutines
A coroutine is a specialized version of a Python `generator` function.
> A coroutine can suspend its execution before reaching return

In [9]:
import asyncio # use asyncio
async def c():
    print(f"Start...")
    await asyncio.sleep(1)
    print(f"End...")
async def main():
    await asyncio.gather(c(),c(),c(),c())
if __name__ == "__main__":
    import time
    s = time.perf_counter()
    await main() # asyncio.run(main()) # only for standalone
    elapsed = time.perf_counter() - s
    print(f"{__name__} executed in {elapsed:4f} seconds.") 

Start...
Start...
Start...
Start...
End...
End...
End...
End...
__main__ executed in 1.003738 seconds.


In [12]:
def c():
    print(f"Start...")
    time.sleep(1) # Note: can't just await for asyncio, c() is **NOT** async
    print(f"End...")
def main():
    for _ in range(4):
        c()
if __name__ == "__main__":
    import time
    s = time.perf_counter()
    main()
    elapsed = time.perf_counter() - s
    print(f"{__name__} executed in {elapsed:4f} seconds.") 

Start...
End...
Start...
End...
Start...
End...
Start...
End...
__main__ executed in 4.007882 seconds.


- `async def` results in a **native coroutine** or an **asynchronous generator** (`async with` and `async for` are valid as well).
- `await` hands function control over the event loop (*by suspending the associated `surrounding` coroutine*). 
> **To Clarify:** <br>
\- If an *`await f()`* is encountered within the scope of *`g()`*, the execution of `g()` is suspended till the result of `f()` is returned. <br>
\- Something else meanwhile runs
```python
async def g():
    \# Pause and back to g() when f() is ready
    await f()
    return r
```

In [18]:
import asyncio
import random
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
) # ANSI colors
async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = random.randint(0, 10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = random.randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i

async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res

if __name__ == "__main__":
    random.seed(444)
    r1, r2, r3 = await main() # r1, r2, r3 = asyncio.run(main())
    print()
    print(f"r1: {r1}, r2: {r2}, r3: {r3}")

Initiated makerandom(0).
makerandom(0) == 4 too low; retrying.
Initiated makerandom(1).
makerandom(1) == 4 too low; retrying.
Initiated makerandom(2).
makerandom(2) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 8 too low; retrying.
---> Finished: makerandom(1) == 10
makerandom(0) == 7 too low; retrying.
makerandom(0) == 8 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 7 too low; retrying.
makerandom(0) == 1 too low; retrying.
makerandom(0) == 6 too low; retrying.
---> Finished: makerandom(2) == 9
makerandom(0) == 3 too low; retrying.
makerandom(0) == 9 too low; retrying.
makerandom(0) == 7 too low; retrying.
---> Finished: makerandom(0) == 10

r1: 10, r2: 10, r3: 9


In [21]:
print (u"\u001b[40m A \u001b[41m B \u001b[42m C \u001b[43m D \u001b[0m")
print (u"\u001b[44m A \u001b[45m B \u001b[46m C \u001b[47m D \u001b[0m")
print (u"\u001b[40;1m A \u001b[41;1m B \u001b[42;1m C \u001b[43;1m D \u001b[0m")
print (u"\u001b[44;1m A \u001b[45;1m B \u001b[46;1m C \u001b[47;1m D \u001b[0m")

 A  B  C  D 
 A  B  C  D 
 A  B  C  D 
 A  B  C  D 


In [22]:
import sys
for i in range(0, 16):
    for j in range(0, 16):
        code = str(i * 16 + j)
        sys.stdout.write(u"\u001b[48;5;" + code + "m " + code.ljust(4))
    print (u"\u001b[0m")

 0    1    2    3    4    5    6    7    8    9    10   11   12   13   14   15  
 16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31  
 32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47  
 48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63  
 64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79  
 80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95  
 96   97   98   99   100  101  102  103  104  105  106  107  108  109  110  111 
 112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127 
 128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143 
 144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159 
 160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175 
 176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191 
 192  193  194  195  196  19

In [36]:
print("\033[35m"f"test")

test


# Async IO patterns
## Chaining Corroutines
A coroutine object is awaitable, so another coroutine can `await` -> smaller manageable recyclable .....

In [ ]:
import asyncio
import random
import time

async def part1(n: int) -> str:
    i = random.randint(0, 10)
    print(f"part1({n}) sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-1"
    print(f"Returning part1({n}) == {result}.")
    return result

async def part2(n: int, arg: str) -> str:
    i = random.randint(0, 10)
    print(f"part2{n, arg} sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-2 derived from {arg}"
    print(f"Returning part2({n, arg}) == {result}.")
    return result

async def chain(n: int) -> None:
    start = time.perf_counter()
    p1 = await part1(n)
    p2 = await part2(n, p1)
    end = time.perf_counter() - start
    print(f"-->Chained result{n} => {p2} (took {end} seconds).")

async def main(*args):
    await asyncio.gather(*(chain(n) for n in args))

if __name__ == "__main__":
    import sys
    random.seed(444)
    args = [1, 2, 3] if len(sys.argv) == 1 else map(int, sys.argv[1:])
    start = time.perf_counter()
    asyncio.run(main(*args))
    end = time.perf_counter() - start
    print(f"Program finished in {end} seconds.")

In [2]:
import asyncio
import itertools as it
import os
import random
import time

async def makeitem(size: int = 5) -> str:
    return os.urandom(size).hex()

async def randsleep(caller=None) -> None:
    i = random.randint(0, 10)
    if caller:
        print(f"{caller} sleeping for {i} seconds.")
    await asyncio.sleep(i)

async def produce(name: int, q: asyncio.Queue) -> None:
    n = random.randint(0, 10)
    for _ in it.repeat(None, n):  # Synchronous loop for each single producer
        await randsleep(caller=f"Producer {name}")
        i = await makeitem()
        t = time.perf_counter()
        await q.put((i, t))
        print(f"Producer {name} added <{i}> to queue.")

async def consume(name: int, q: asyncio.Queue) -> None:
    while True:
        await randsleep(caller=f"Consumer {name}")
        i, t = await q.get()
        now = time.perf_counter()
        print(f"Consumer {name} got element <{i}>"
              f" in {now-t:0.5f} seconds.")
        q.task_done()

async def main(nprod: int, ncon: int):
    q = asyncio.Queue()
    producers = [asyncio.create_task(produce(n, q)) for n in range(nprod)]
    consumers = [asyncio.create_task(consume(n, q)) for n in range(ncon)]
    await asyncio.gather(*producers)
    await q.join()  # Implicitly awaits consumers, too
    for c in consumers:
        c.cancel()

if __name__ == "__main__":
    import argparse
    random.seed(444)
    parser = argparse.ArgumentParser()
    parser.add_argument("-p", "--nprod", type=int, default=5)
    parser.add_argument("-c", "--ncon", type=int, default=10)
    ns = parser.parse_args()
    start = time.perf_counter()
    asyncio.run(main(**ns.__dict__))
    elapsed = time.perf_counter() - start
    print(f"Program completed in {elapsed:0.5f} seconds.")

usage: ipykernel_launcher.py [-h] [-p NPROD] [-c NCON]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-437d860e-0ba0-4d31-af12-aca038a72b98.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
loop = asyncio.get_event_loop()